In [ ]:
api_key = ""

In [ ]:
%pip install httpx
%pip install numpy
%pip install pandas
%pip install plotly
%pip install "nbformat>=4.2.0"

In [ ]:
import json
import random
import time
from datetime import datetime
from datetime import timedelta
from math import cos, sin, pi, sqrt, radians, asin

import httpx
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
max_weight = 4
number_packages = 150
center_lon = 16.3728
center_lat = 48.2087
delivery_start_time = datetime.now()

parameters = {
    "max_capacity": 15,
    "max_time_in_minutes": 180,
    "min_profit_per_hour": 12.0,
    "max_profit_per_hour": 26.0,
    "pick_loading_time_minutes": 3,
    "drop_loading_time_minutes": 2,
    "avg_shipment_per_vehicle": 5,
    "delivery_start_time": delivery_start_time.isoformat(),
}

In [ ]:
def rand_lon_lat(center_lon, center_lat, angle_norm, distance_norm, distance_scale=0.06):
    distance = sqrt(distance_norm) * distance_scale
    angle = angle_norm * 2 * pi
    lat = center_lat + distance * sin(angle)
    lon = center_lon + distance * cos(angle) * 111 / 74  # correction (values around Vienna)
    return lon, lat

def great_circle(lon1, lat1, lon2, lat2, r=6371):
    """
    Calculate the great circle distance (haversine)
    between two points specified in decimal degrees
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    km = r * c
    return km * 1000  # return in meter

packages = []
for i in range(number_packages):
    uuid = i
    src_lon, src_lat = rand_lon_lat(center_lon, center_lat, random.random(), random.random())
    dst_lon, dst_lat = rand_lon_lat(center_lon, center_lat, random.random(), random.random())
    weight_kg = int(round(random.random() * max_weight, 0))
    dist_meter = great_circle(src_lon, src_lat, dst_lon, dst_lat) * 1.2  # use air distance in demo
    profit = (  # simple price based on distance and weight
        2.10 +  # 2.1 EUR base price
        dist_meter // 100 * 0.08 +  # each 100 meter is 8 cent
        weight_kg // 2  # 0-1 kg 0 EUR, 2-3 kg 1 EUR, 4-5 2 EUR
    )  
    size = weight_kg + 1
    package = [
        uuid,
        src_lon,
        src_lat,
        dst_lon,
        dst_lat,
        profit,
        size,
        parameters["pick_loading_time_minutes"],
        parameters["drop_loading_time_minutes"],
        delivery_start_time,  # pick_from_time
        delivery_start_time,  # drop_from_time
        delivery_start_time + timedelta(minutes=parameters["max_time_in_minutes"]),  # pick_until_time
        delivery_start_time + timedelta(minutes=parameters["max_time_in_minutes"]),  # drop_until_time
    ]
    packages.append(package)

packages = pd.DataFrame(
    packages,
    columns=[
        "shipment_id",
        "pick_longitude",
        "pick_latitude",
        "drop_longitude",
        "drop_latitude",
        "profit",
        "size",
        "pick_loading_time_minutes",
        "drop_loading_time_minutes",
        "pick_from_time",
        "drop_from_time",
        "pick_until_time",
        "drop_until_time",
    ]
)

smartcity_api_data = {
        "api_key": api_key,
        "parameters": parameters,
        "shipments": [v for k, v in json.loads(packages.to_json(orient="index")).items()],
}

In [ ]:
response = httpx.post('https://www.smartcityroutes.com/api/v1/optimize', json=smartcity_api_data)
if response.is_error:
    print(response.json())
    raise RuntimeError("Failed to send data")
request_id = response.json()["request_id"]
print("Started computation with request ID %s" % request_id)

In [ ]:
status_last = ""
progress_last = 0
while True:
    response = httpx.get('https://www.smartcityroutes.com/api/v1/status/request_id=%s' % request_id)
    if response.is_error:
        print(response.json())
        raise RuntimeError("Failed to query results")
    response = response.json()
    status = response["status_str"]

    if status_last != status:
        status_last = status
        print("Status at %s: %s" % (datetime.now().isoformat(), status))
        if status == "waiting":
            print(
                """Our current version is in Alpha state.\n"""
                """In this step the script is starting the computers in the cloud.\n"""
                """This can take up to 2 mintes.\n"""
                """In a production environment the machines would already be ready and waiting for data."""
            )

    if status == "computing":
        progress = response["progress"]
        if progress_last != progress:
            progress_last = progress
            print("Current progress: %d" % progress)

    if status == "finished" or status == "error":
        break
    else:
        time.sleep(10)

In [ ]:
def parse_output(results: pd.DataFrame):
    data = [
        pd.concat([results["shipment_id"], results["shipment_id"]], ignore_index=True),
        pd.concat([results["vehicle_uuid"], results["vehicle_uuid"]], ignore_index=True),
        pd.concat([results["pick_time"], results["drop_time"]], ignore_index=True),
        pd.DataFrame(["pick"] * len(results) + ["drop"] * len(results)),
    ]
    routes = pd.concat(data, axis=1)
    routes.columns = ["shipment_id", "vehicle_uuid", "time", "pick/drop"]
    routes["time"] = pd.to_datetime(routes["time"])
    return routes

#print(smartcity_api_data)
#print(response["shipments"])
results = pd.DataFrame(response["shipments"])
results["shipment_id"] = results["shipment_id"].astype(int)
results = parse_output(results)
packages_twice = pd.concat([packages, packages])
packages_twice["latitude"] = pd.concat([packages["pick_latitude"], packages["drop_latitude"]])
packages_twice["longitude"] = pd.concat([packages["pick_longitude"], packages["drop_longitude"]])
packages_twice["pick/drop"] = ["pick"] * len(packages) + ["drop"] * len(packages)
packages_twice["pick/drop_size"] = [2] * len(packages) + [1] * len(packages)
packages_done = packages_twice.merge(results, right_on=["shipment_id", "pick/drop"], left_on=["shipment_id", "pick/drop"], how="outer")
packages_done = packages_done.fillna(value='None')
packages_done = packages_done.sort_values(['vehicle_uuid', 'time'])

In [ ]:
# plot package pick and drop locations
fig_1 = px.scatter_mapbox(packages_done, lat='latitude', lon='longitude', size='pick/drop_size',
                        hover_data=packages_done.columns,#{'pick_latitude': ':.4f', 'pick_longitude': ':.4f', 'shipment_id': True},
                        hover_name=None, color='vehicle_uuid', zoom=11, height=1200, center=dict(lat=center_lat, lon=center_lon),
)

# fig_1.update_traces(
#     marker=dict(size=8, symbol="diamond"),
#     selector=dict(mode="markers"),
# )

# plot delivery routes
fig_2 = px.line_mapbox(packages_done, lat='latitude', lon='longitude', color="vehicle_uuid", zoom=11, height=1200, center=dict(lat=center_lat, lon=center_lon))

# merge two plots
fig = go.Figure(data=fig_1.data + fig_2.data)

# google colab does not appriciate this
# hide all except one route
# fig.for_each_trace(
#     lambda trace: trace.update(visible="legendonly")
#     if trace.name != packages_done["vehicle_uuid"][0] else ())

# configure layout of plot
fig.update_layout(
    height = 1000,
    margin = {"r": 5, "t": 5, "l": 5, "b": 5},
    mapbox = {
        'center': {'lon': center_lon, 'lat': center_lat},
        'style': "carto-positron",
        #'style': "stamen-terrain",
        'zoom': 11,
    }
)

fig.show()